In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install -U tensorflow-addons~=0.7.0
!pip install -U tensorflow
!pip install -U keras

     |████████████████████████████████| 993kB 2.8MB/s 
     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 3.9MB 38.4MB/s 
     |████████████████████████████████| 450kB 60.9MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.1.0)
     |████████████████████████████████| 378kB 2.7MB/s 
  Found existing installation: Keras 2.2.5

In [0]:
# only colab code

In [0]:
# 압축파일 옮기기
!mkdir -p /content/kaggle/working/dataset
!cp /content/drive/'My Drive'/kaggle/working/dataset/REAL2.zip /content/kaggle/working/dataset/
!cp /content/drive/'My Drive'/kaggle/working/dataset/FAKE2.zip /content/kaggle/working/dataset/

In [0]:
!unzip -q /content/kaggle/working/dataset/REAL2.zip -d /content/kaggle/working/dataset/REAL2
!unzip -q /content/kaggle/working/dataset/FAKE2.zip -d /content/kaggle/working/dataset/FAKE2

In [0]:
import os
os.getcwd()
os.chdir('/content/kaggle/working')

In [0]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [0]:
# 전처리한 이미지 불러오기
input_shape = (75, 75, 3)
data_dir = '../working/dataset'

real_data = [data_dir + '/REAL2/'+ f for f in os.listdir(data_dir+'/REAL2') if f.endswith('.png')]
fake_data = [data_dir + '/FAKE2/'+ f for f in os.listdir(data_dir+'/FAKE2') if f.endswith('.png')]

# fake_data > real_data
real_data = np.random.choice(real_data, 50000, replace=False)
fake_data = np.random.choice(fake_data, 50000, replace=False)
X = []
Y = []

# 이미지 불러서 list로 저장
for idx, img in enumerate(fake_data):
    X.append(cv2.resize(img_to_array(load_img(img)),(75, 75)) / 255.0)
    Y.append(0)
    if (idx+1) % 5000 == 0:
        print(idx+1)

for idx, img in enumerate(real_data):
    X.append(cv2.resize(img_to_array(load_img(img)),(75, 75)) / 255.0) # .flatten()
    Y.append(1)
    if (idx+1) % 5000 == 0:
        print(idx+1)


5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [0]:
#Y_val_org = Y

#Normalization
X = np.array(X)
Y = to_categorical(Y, 2)

In [0]:
X.shape

(100000, 75, 75, 3)

In [0]:
#Reshape
#X = X.reshape(-1, 75, 75, 3)

#Train-Test split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

In [0]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [0]:
googleNet_model = InceptionResNetV2(include_top=False, input_shape=input_shape) #, weights='imagenet'
googleNet_model.trainable = True
model = Sequential()
model.add(googleNet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=2, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1, 1, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1536)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 3074      
Total params: 54,339,810
Trainable params: 54,279,266
Non-trainable params: 60,544
_________________________________________________________________


In [0]:
early_stopping = EarlyStopping(monitor='val_loss', #val_accuracy
                               min_delta=0,
                               patience=3,
                               verbose=0, mode='auto')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999), #lr=1e-5, beta_1=0.9, beta_2=0.99
              metrics=['accuracy'])

EPOCHS = 100
BATCH_SIZE = 128
history = model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = (X_val, Y_val), verbose = 1,  callbacks=[early_stopping]) #, mc

Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 160s 2ms/sample - loss: 0.4615 - accuracy: 0.7915 - val_loss: 0.4186 - val_accuracy: 0.8256
Epoch 2/100
80000/80000 [==============================] - 139s 2ms/sample - loss: 0.3808 - accuracy: 0.8359 - val_loss: 0.3847 - val_accuracy: 0.8385
Epoch 3/100
80000/80000 [==============================] - 139s 2ms/sample - loss: 0.3464 - accuracy: 0.8515 - val_loss: 0.3653 - val_accuracy: 0.8480
Epoch 4/100
80000/80000 [==============================] - 140s 2ms/sample - loss: 0.3147 - accuracy: 0.8666 - val_loss: 0.3542 - val_accuracy: 0.8566
Epoch 5/100
80000/80000 [==============================] - 139s 2ms/sample - loss: 0.2808 - accuracy: 0.8831 - val_loss: 0.3359 - val_accuracy: 0.8640
Epoch 6/100
80000/80000 [==============================] - 139s 2ms/sample - loss: 0.2463 - accuracy: 0.8996 - val_loss: 0.3313 - val_accuracy: 0.8727
Epoch 7/100
80000/80000 [===================

In [0]:
# loss, 정확도 시각화
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
t = f.suptitle('Pre-trained InceptionResNetV2 Transfer Learn with Fine-Tuning & Image Augmentation Performance ', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

EPOCHS = early_stopping.stopped_epoch + 1

epoch_list = list(range(1,EPOCHS+1))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, EPOCHS+1, 1))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch #')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, EPOCHS+1, 1))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch #')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [0]:
model.save('breath-model.h5')

In [0]:
# from google.colab import files
# files.download('breath-model.h5')

In [0]:
!cp /content/kaggle/working/breath-model.h5 /content/drive/'My Drive'/kaggle/